**Channel Grouping**
𝑠^[𝑙]   : Channels in the left hemisphere
𝑠^[𝑟]   : Corresponding channels in the right hemisphere
𝑠^[𝑚]   : Channels in the midline region

**Hemispheric Difference Computation**
For each pair of corresponding left and right hemisphere channels (j ∈ [1,〖 𝑙〗_𝑔]), calculate the difference.
ⅆ_𝑗=𝑠_𝑖^[𝑙] −𝑠_𝑗^[𝑟] 

**Final Feature Construction**
The set of all hemispheric differences (D = [ⅆ_𝑗]) is concatenated with midline signals 𝑠^[𝑚] , and the final feature vector is computed as:
𝑋=[𝐷^𝑇 𝑠^[𝑚]𝑇 ]

In [ ]:
import torch
import numpy as np

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Normalize labels to [0, 1, 2, 3]
y_train = first_session_labels - np.min(first_session_labels)
y_test = second_session_labels - np.min(second_session_labels)

print(f"First session data shape: {first_session_data.shape}")
print(f"Second session data shape: {second_session_data.shape}")

# EEG channel renaming
EEG_CH_NAMES = [          
    "Fz","FC3","FC1","FCz","FC2","FC4",
    "C5","C3","C1","Cz","C2","C4","C6",
    "CP3","CP1","CPz","CP2","CP4",
    "P1","Pz","P2","POz"
]

LEFT_CHS     = ["FC3", "FC1", "C5", "C3", "C1", "CP3", "CP1", "P1"]
RIGHT_CHS    = ["FC4", "FC2", "C6", "C4", "C2", "CP4", "CP2", "P2"]
MIDLINE_CHS  = ["Fz", "FCz", "Cz", "CPz", "Pz", "POz"]

def get_channel_indices(ch_names, full_list):
    return [full_list.index(ch) for ch in ch_names]

idx_l = get_channel_indices(LEFT_CHS, EEG_CH_NAMES)
idx_r = get_channel_indices(RIGHT_CHS, EEG_CH_NAMES)
idx_m = get_channel_indices(MIDLINE_CHS, EEG_CH_NAMES)

# Region-level feature extraction
def extract_region_features(data):
    D = data[:, idx_l, :] - data[:, idx_r, :]
    M = data[:, idx_m, :]
    return np.concatenate([D, M], axis=1)

# Apply region-level extraction
X_first_region = extract_region_features(first_session_data)
X_second_region = extract_region_features(second_session_data)

print(f"Region-level feature shape: {X_first_region.shape}")

# Robust normalization
def robust_normalize(data):
    normalized_data = np.zeros_like(data)
    for trial in range(data.shape[0]):
        for channel in range(data.shape[1]):
            channel_data = data[trial, channel, :]
            median = np.median(channel_data)
            mad = np.median(np.abs(channel_data - median))
            if mad > 0:
                normalized_data[trial, channel, :] = (channel_data - median) / (1.4826 * mad)
            else:
                normalized_data[trial, channel, :] = channel_data - median
    return normalized_data

X_first_session = robust_normalize(X_first_region)
X_second_session = robust_normalize(X_second_region)
